In [3]:
from __future__ import division
from __future__ import absolute_import
from __future__ import print_function
import tensorflow as tf 
from keras.engine.topology import Layer
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
import keras.backend as K
import numpy as np
from keras.datasets import imdb
import preprocessing
from sklearn.metrics import accuracy_score

D:\Miniconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [28]:
import importlib
importlib.reload(preprocessing)

<module 'preprocessing' from 'D:\\jupyter-workdir\\nlp\\bg_fusion\\preprocessing.py'>

In [29]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(path="E:/paper/stackingmodel/imdb/imdb.npz", num_words=10000)

In [71]:
[X_train_cut, X_test_cut], (min_len, max_len) = preprocessing.auto_padding([X_train, X_test], sentence_size=300)

In [96]:
def get_spp_embeding_model(num_words, mode = "avg", word_num_per_sent = 100, word_size = 100):
    word_indices = Input(shape=[word_num_per_sent], name="word_indices")
    word_embedding = Embedding(num_words, word_size)(word_indices)
    x_flow = word_embedding
    x_flow = Dropout(0.1)(word_embedding)
    x_flow = Conv1D(256, 3, padding='same', activation='relu', strides=1)(x_flow)
    x_flow = MaxPooling1D(3)(x_flow)
    x_flow = Flatten()(x_flow)
    x_flow = Dense(word_size, activation='relu')(x_flow)
    y_output = Dense(1, activation='sigmoid')(x_flow)
    sgd = Adam(lr=1e-3)
    model = Model(inputs=[word_indices], outputs=y_output)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [97]:
embed_model = get_spp_embeding_model(num_words=10000, word_size=200, word_num_per_sent=300)
embed_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_indices (InputLayer)    (None, 300)               0         
_________________________________________________________________
embedding_15 (Embedding)     (None, 300, 200)          2000000   
_________________________________________________________________
dropout_13 (Dropout)         (None, 300, 200)          0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 300, 256)          153856    
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 100, 256)          0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 25600)             0         
_________________________________________________________________
dense_27 (Dense)             (None, 200)               5120200   
__________

In [98]:
embed_model.fit(X_train_cut, y_train, batch_size=256, epochs=20)

Epoch 1/20
25000/25000 [==============================] - 6s 250us/step - loss: 0.7019 - acc: 0.5156
Epoch 2/20
25000/25000 [==============================] - 5s 209us/step - loss: 0.5995 - acc: 0.6714
Epoch 3/20
25000/25000 [==============================] - 5s 209us/step - loss: 0.4400 - acc: 0.8886
Epoch 4/20
25000/25000 [==============================] - 5s 209us/step - loss: 0.3675 - acc: 0.9204
Epoch 5/20
25000/25000 [==============================] - 5s 210us/step - loss: 0.3155 - acc: 0.9446
Epoch 6/20
25000/25000 [==============================] - 5s 209us/step - loss: 0.2800 - acc: 0.9568
Epoch 7/20
25000/25000 [==============================] - 5s 211us/step - loss: 0.2547 - acc: 0.9628
Epoch 8/20
25000/25000 [==============================] - 5s 210us/step - loss: 0.2351 - acc: 0.9677
Epoch 9/20
25000/25000 [==============================] - 5s 210us/step - loss: 0.2197 - acc: 0.9695
Epoch 10/20
25000/25000 [==============================] - 5s 210us/step - loss: 0.2099 - a

In [99]:
test_y_hat = embed_model.predict(X_test_cut)
accuracy_score(y_test, np.round(test_y_hat + 1e-5))

0.85324